In [ ]:
pip install -q tensorflow_datasets

In [ ]:
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
dataset,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)
train_data,test_data=dataset['train'],dataset['test']
train_data.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXKZLCY/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXKZLCY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXKZLCY/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
print('train_len {}'.format(len(train_data)),'test_len {}'.format(len(test_data)))

train_len 25000 test_len 25000


In [ ]:
for text,label in train_data.take(5):
  print('text:',text.numpy())
  print('label:',label.numpy())


text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label: 0
text: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The

In [ ]:
BUFFER_SIZE=10000
BATCH_SIZE=64

In [ ]:
train_data=train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data=test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for text,label in train_data.take(1):
  print("text:",text.numpy()[:3])
  print('label:',label.numpy()[:3])

text: [b'Revolt of the Zombies is BAD. There is nothing remotely entertaining about the movie. It is dull, lifeless, poorly acted, and poorly scripted. I\'ve often complained that the original Dracula is a little slow for my taste, well this movie makes Dracula look like a roller coaster ride. The 65 minute running time seemed like 165 minutes.<br /><br />The story: An expedition is sent to Cambodia to find the secrets of mind control through "zombification". One man finds the secret and uses it to make the woman he loves marry him. Once this happens, he releases the zombies under his control to horrific consequences. That\'s it. That\'s the whole story.<br /><br />For most of the movie, I was trying to figure out where I had seen the male lead. He looked so familiar. I had plenty of time to think this over. Nothing was happening in the movie. Just before the "zombies revolted", it hit me. It was Dean Jagger. I had seen him recently as the General in White Christmas. This is how I "ent

In [ ]:
encoder=tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=1000)
encoder.adapt(train_data.map(lambda text,lebel:text))

In [ ]:
vocabulary=np.array(encoder.get_vocabulary())
vocabulary[:25]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but',
       'film', 'on', 'not', 'you', 'are'], dtype='<U14')

In [ ]:
encoded_example = encoder(text)[:3].numpy()
encoded_example

array([[  1,   5,   2, ...,   0,   0,   0],
       [  1,   1,   3, ...,   0,   0,   0],
       [ 74,   1, 154, ...,   0,   0,   0]])

With the default settings, the process is not completely reversible

The limited vocabulary size and lack of character-based fallback results in some unknown tokens.

In [ ]:
for n in range(3):
  print("Original: ", text[n].numpy())
  print("Round-trip: ", " ".join(vocabulary[encoded_example[n]]))
  print()

Original:  b'Revolt of the Zombies is BAD. There is nothing remotely entertaining about the movie. It is dull, lifeless, poorly acted, and poorly scripted. I\'ve often complained that the original Dracula is a little slow for my taste, well this movie makes Dracula look like a roller coaster ride. The 65 minute running time seemed like 165 minutes.<br /><br />The story: An expedition is sent to Cambodia to find the secrets of mind control through "zombification". One man finds the secret and uses it to make the woman he loves marry him. Once this happens, he releases the zombies under his control to horrific consequences. That\'s it. That\'s the whole story.<br /><br />For most of the movie, I was trying to figure out where I had seen the male lead. He looked so familiar. I had plenty of time to think this over. Nothing was happening in the movie. Just before the "zombies revolted", it hit me. It was Dean Jagger. I had seen him recently as the General in White Christmas. This is how I 

In [ ]:
model=keras.Sequential([
                        encoder,
                        layers.Embedding(input_dim=len(encoder.get_vocabulary()),output_dim=64,mask_zero=True),
                        layers.Bidirectional(layers.LSTM(64)),
                        layers.Dense(64,activation='relu'),
                        layers.Dense(1)

])

model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,validation_data=test_data,epochs=5)

Epoch 1/5
391/391 [==============================] - 55s 106ms/step - loss: 0.6747 - accuracy: 0.5263 - val_loss: 0.4371 - val_accuracy: 0.7876
Epoch 2/5
391/391 [==============================] - 40s 102ms/step - loss: 0.4091 - accuracy: 0.8109 - val_loss: 0.3436 - val_accuracy: 0.8477
Epoch 3/5
391/391 [==============================] - 40s 100ms/step - loss: 0.3323 - accuracy: 0.8538 - val_loss: 0.3253 - val_accuracy: 0.8499
Epoch 4/5
391/391 [==============================] - 40s 101ms/step - loss: 0.3192 - accuracy: 0.8598 - val_loss: 0.3198 - val_accuracy: 0.8553
Epoch 5/5
391/391 [==============================] - 40s 101ms/step - loss: 0.3186 - accuracy: 0.8627 - val_loss: 0.3245 - val_accuracy: 0.8622


In [ ]:
test_loss,test_acc=model.evaluate(test_data)
print('test_loss {}'.format(test_loss))
print('test_acc{}'.format(test_acc))

391/391 [==============================] - 13s 33ms/step - loss: 0.3245 - accuracy: 0.8622
test_loss 0.3244667947292328
test_acc0.8622000217437744


In [ ]:
sample_text=('the story line normal,but the screenplay was so strong,graphics and music is awesome overall it is a good movie ')
prediction=model.predict(np.array([sample_text]))
print(prediction[0])

[0.8130478]
